In [67]:

import os
from pathlib import Path
import json
import csv
import pickle


def path_to_list(path):
    path = Path(path)
    print(str(path))
    if not path.exists():
        raise FileNotFoundError(f'Директория {path} не найдена')
    if not path.is_dir():
        raise NotADirectoryError(f'{path} - не директория')
    
    path_list = []
    for file in path.iterdir():
        if file.is_file():
            path_list.append({'name': file.name, 
                              'path': os.path.abspath(file), 
                              'type': 'File', 
                              'size': os.path.getsize(os.path.abspath(file)), 
                              'parent_dir': str(path)})
        if file.is_dir():
            path_list.append({'name': file.name, 
                              'path': os.path.abspath(file), 
                              'type': 'Dir', 
                              'size': sum(os.path.getsize(elem) for elem in os.scandir(file)) , 
                              'parent_dir': str(path)})

    return path_list


def path_to_json(path_list):
    with open('path_to_json.json', 'w', encoding='utf-8') as f:
        json.dump(path_list, f, indent=4)


def path_to_csv(path_list):
    with open('path_to_csv.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['name', 'path', 'type', 'size', 'parent_dir'])
        for i in path_list:
            writer.writerow([i['name'], i['path'], i['type'], i['size'], i['parent_dir']])


def save_to_csv(dict_list, file_name='save_to_csv.csv'):
    with open(file_name, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        # writer.writerow(['name', 'path', 'type', 'size', 'parent_dir'])
        header = list(dict_list[0].keys())
        writer.writerow(header)
        for i in dict_list:
            # writer.writerow([i['name'], i['path'], i['type'], i['size'], i['parent_dir']])
            writer.writerow(list(i[head] for head in header))


def path_to_pickle(path_list):
    with open('path_to_pickle.pkl', 'wb') as f:
        pickle.dump(path_list, f)


def main():
    path = input('Введите путь к директории: ')
    path_list = path_to_list(path)    
    path_to_json(path_list)
    save_to_csv(path_list, 'path_to_csv.csv')
    path_to_pickle(path_list)

# print(*path_List("k:/Мой диск/GeekBrains/Знакомство с языками программирования/Python/Python_Immersion"), sep='\n')

path_list = path_to_list(".")
print(*path_list, sep='\n')
path_to_json(path_list)
save_to_csv(path_list, 'path_to_csv1.csv')
# path_to_csv(path_list)
path_to_pickle(path_list)




.
{'name': 'README.md', 'path': 'k:\\Мой диск\\GeekBrains\\Знакомство с языками программирования\\Python\\Python_Immersion\\homework8\\README.md', 'type': 'File', 'size': 1125, 'parent_dir': '.'}
{'name': 'task2.py', 'path': 'k:\\Мой диск\\GeekBrains\\Знакомство с языками программирования\\Python\\Python_Immersion\\homework8\\task2.py', 'type': 'File', 'size': 2946, 'parent_dir': '.'}
{'name': 'path_to_json.json', 'path': 'k:\\Мой диск\\GeekBrains\\Знакомство с языками программирования\\Python\\Python_Immersion\\homework8\\path_to_json.json', 'type': 'File', 'size': 6034, 'parent_dir': '.'}
{'name': 'path_to_csv.csv', 'path': 'k:\\Мой диск\\GeekBrains\\Знакомство с языками программирования\\Python\\Python_Immersion\\homework8\\path_to_csv.csv', 'type': 'File', 'size': 2319, 'parent_dir': '.'}
{'name': 'path_to_pickle.pkl', 'path': 'k:\\Мой диск\\GeekBrains\\Знакомство с языками программирования\\Python\\Python_Immersion\\homework8\\path_to_pickle.pkl', 'type': 'File', 'size': 2499, 'pa

In [49]:
# Задание 2.
# Напишите функцию, которая получает на вход директорию и рекурсивно обходит её и все вложенные директории. Результаты обхода сохраните в файлы json, csv и pickle.
# - Для дочерних объектов указывайте родительскую директорию.
# - Для каждого объекта укажите файл это или директория.
# - Для файлов сохраните его размер в байтах, а для директорий размер файлов в ней с учётом всех вложенных файлов и директорий.

from pathlib import Path
import os, csv, json, pickle
import pprint


def save_dir(path: Path = Path.cwd()) -> None:
    if path.is_dir():
        data = dir_rec(path)
        pprint.pprint(data, indent=2)
        save_to_json(data, 'hw8_result.json')
        save_to_csv(data, 'hw8_result.csv')
        save_to_pickle(data, 'hw8_result.pkl')
    else:
        print('Это не директория!')

def dir_rec(path: Path):
    data = []
    path = Path(path)
    for file in path.iterdir():
        dir_size = 0


        if file.is_dir():
            dir_path = os.path.abspath(file)
            dir_size += get_dir_size(dir_path)
            data.append({
                'name': file.name,
                'path': dir_path,
                'type': 'directory',
                'size': dir_size,
                'parent_directory': str(path),
            })
            data += dir_rec(Path(dir_path))

        if file.is_file():
            file_path = os.path.abspath(file)
            file_size = os.path.getsize(file_path)
            dir_size += file_size
            data.append({
                'name': file.name,
                'path': file_path,
                'type': 'file',
                'size': file_size,
                'parent_directory': str(path)
            })
    print(*data)

    return data

def get_dir_size(path):
    total_size = 0

    for path, dirs, files in os.walk(path):
        for file in files:
            f_path = os.path.join(path, file)
            total_size += os.path.getsize(f_path)

    return total_size


def save_to_json(data, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        json.dump(data, file, indent=4)


def save_to_csv(data, filename):
    headers = ['name','path', 'type', 'size', 'parent_directory']
    with open(filename, 'w', encoding='utf-8', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        writer.writeheader()
        writer.writerows(data)


def save_to_pickle(data, filename):
    with open(filename, 'wb') as file:
        pickle.dump(data, file)


# if __name__ == '__main__':
#     # save_dir(Path('C:\\Nata\\GeekBrains\\gb-git\\python_adv\\seminar7'))
#     save_dir()

save_dir(Path('.'))

{'name': 'README.md', 'path': 'k:\\Мой диск\\GeekBrains\\Знакомство с языками программирования\\Python\\Python_Immersion\\homework8\\README.md', 'type': 'file', 'size': 1125, 'parent_directory': '.'} {'name': 'task2.py', 'path': 'k:\\Мой диск\\GeekBrains\\Знакомство с языками программирования\\Python\\Python_Immersion\\homework8\\task2.py', 'type': 'file', 'size': 2870, 'parent_directory': '.'} {'name': 'proba8.ipynb', 'path': 'k:\\Мой диск\\GeekBrains\\Знакомство с языками программирования\\Python\\Python_Immersion\\homework8\\proba8.ipynb', 'type': 'file', 'size': 109971, 'parent_directory': '.'} {'name': 'path_to_json.json', 'path': 'k:\\Мой диск\\GeekBrains\\Знакомство с языками программирования\\Python\\Python_Immersion\\homework8\\path_to_json.json', 'type': 'file', 'size': 7686, 'parent_directory': '.'} {'name': 'path_to_csv.csv', 'path': 'k:\\Мой диск\\GeekBrains\\Знакомство с языками программирования\\Python\\Python_Immersion\\homework8\\path_to_csv.csv', 'type': 'file', 'size

In [63]:
test = {'key1': 'value1', 'key2': 'value2'}
keys_list = list(test.keys())
print(keys_list)

['key1', 'key2']
